In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from itertools import count
from selenium import webdriver

import ssl

def get_request_url(url, enc='utf-8'):
    
    req = urllib.request.Request(url)

    try:
        ssl._create_default_https_context = ssl._create_unverified_context

        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            try:
                rcv = response.read()
                ret = rcv.decode(enc)
            except UnicodeDecodeError:
                ret = rcv.decode(enc, 'replace')    
            
            return ret
            
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

def CarAddress(result):
    wd = webdriver.Edge('./msedgedriver.exe')
    wd.get("http://www.xn--vk1b209baa037j.kr/main.php?m=sale&s=list&carnation=1")
    
    for page_idx in count():
        if page_idx==500: break
        
        print("page count = %s" %str(page_idx + 1)) 
        wd.execute_script("page('%s')" %str(page_idx + 1))
        print("Current Page [%s] Called" % (str(page_idx + 1)))

        rcv_data = wd.page_source
        soupData = BeautifulSoup(rcv_data, 'html.parser')

        soupData_section = soupData.findAll('div', attrs={'class':'txt'})
        for car_list in soupData_section:
            car = list(car_list.strings)
            
            brand = car[1]
            brand = brand.replace("제네시스", "1")
            brand = brand.replace("기아", "2")
            brand = brand.replace("현대", "3")
            brand = brand.replace("쉐보레", "4")
            brand = brand.replace("쌍용", "5")
            brand = brand.replace("삼성", "6")
            brand = brand.replace("대우", "7")
            name = car[2]
            ym, km, trans = car[4].split('\xa0ㆍ\xa0')
            ym = ym[0:4]
            km = km.replace("\xa0","")
            km = km.replace("KM","")
            km = km.replace(",","")
            trans = trans.replace("수동", "0")
            trans = trans.replace("오토", "1")
            price = car[8].split()[0]
            price = price.replace(",","")
            
            result.append([brand, name, ym, km, trans, price])
            print("--------")

    return



def Car():

    result = []

    print('Car ADDRESS CRAWLING START')
    CarAddress(result)
    print(result[0])
    car_table = pd.DataFrame(result, columns=('brand', 'name', 'ym', 'km','trans','price'))
    car_table.to_csv("./car_data.csv", encoding="cp949", mode='w', index=True)
    print('FINISHED')
    
if __name__ == '__main__':
     Car()